# Using EcoFOCIpy to process raw field data

## CTD / Profile Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- Parse data from raw files into pandas dataframe
- output initial files (pandas->csv) **ERDDAP NRT** when no meta data is added

Convert to xarray dataframe for all following work *(working or final data level):
- TODO: Add metadata from cruise yaml files and/or header info
- ingest metadata from cruiese / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:


## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

In [1]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '../'

In [3]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'staticdata/example_data/profile_data/' #<- point to cruise and process all files within
cruise_name = 'd'
cruise_meta_file = sample_data_dir+'staticdata/cruise_example.yaml'
inst_meta_file = sample_data_dir+'staticdata/instr_metaconfig/sbe16_cf.yaml'
inst_shortname = ''
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe9_11p()
filename_list = sorted(glob.glob(datafile + '*.cnv'))

(cruise_data,cruise_header) = cruise.parse(filename_list)

There are 6 files in the example folder ('ctd001.cnv','ctd002.cnv', etc).  The routine will read in all .cnv files in a specified directory and name them based on splitting on "/".  This will work for all CTD 'cnv' files

In [4]:
#example of a single profile
cruise_data['ctd001.cnv']

,c0mS/cm,c1mS/cm,flECO-AFL,sbeox0V,t090C,t190C,timeS,sbeox1V,par,turbWETntu0,...,sigma-t00,sigma-t11,sbeox0ML/L,sbox0Mm/Kg,sbeox0PS,sbeox1ML/L,sbox1Mm/Kg,sbeox1PS,nbin,flag
Pressure [dbar],,,,,,,,,,,,,,,,,,,,,
1.0,29.753597,29.757207,0.4821,2.5202,3.6086,3.6106,112.124,2.5944,304.01000,0.6675,...,25.4895,25.4912,7.3258,319.039,98.191,7.2834,317.191,97.628,15.0,False
2.0,29.755165,29.756555,0.5064,2.5197,3.6097,3.6089,113.162,2.5965,231.14000,0.6915,...,25.4896,25.4916,7.3273,319.103,98.213,7.2978,317.820,97.818,23.0,False
3.0,29.757821,29.759602,0.4824,2.5203,3.6125,3.6125,116.405,2.5987,172.83000,0.6696,...,25.4893,25.4910,7.3337,319.381,98.305,7.3013,317.971,97.873,35.0,False
4.0,29.765491,29.766984,0.5234,2.5208,3.6211,3.6207,149.648,2.6022,136.68000,0.6786,...,25.4886,25.4904,7.3304,319.238,98.282,7.3011,317.964,97.891,37.0,False
5.0,29.764901,29.766794,0.5624,2.5207,3.6198,3.6198,167.871,2.6045,112.94000,0.6606,...,25.4887,25.4905,7.3354,319.457,98.346,7.3068,318.212,97.965,36.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63.0,29.781214,29.783378,0.7888,2.4997,3.6091,3.6094,503.219,2.5851,0.20995,1.2531,...,25.4896,25.4914,7.3121,318.442,98.009,7.3056,318.157,97.923,35.0,False
64.0,29.781581,29.783670,0.8303,2.5250,3.6090,3.6094,506.872,2.5860,0.20996,1.3506,...,25.4897,25.4913,7.4655,325.122,100.064,7.3082,318.270,97.958,53.0,False
65.0,29.782189,29.784315,0.7998,2.5467,3.6093,3.6096,511.657,2.5859,0.20993,1.2990,...,25.4896,25.4914,7.5336,328.086,100.977,7.3031,318.051,97.891,64.0,False


In [5]:
cruise_header['ctd001.cnv']

KeyError: 'ctd001.cnv'

In [6]:
#preview a dataframe
cruise_data['ctd001.cnv'].describe()

,c0mS/cm,c1mS/cm,flECO-AFL,sbeox0V,t090C,t190C,timeS,sbeox1V,par,turbWETntu0,...,sal11,sigma-t00,sigma-t11,sbeox0ML/L,sbox0Mm/Kg,sbeox0PS,sbeox1ML/L,sbox1Mm/Kg,sbeox1PS,nbin
count,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.00000,67.000000,67.000000,67.000000,...,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000
mean,29.769978,29.772047,0.789513,2.488628,3.610676,3.610854,341.44903,2.596190,22.771780,0.778700,...,32.068264,25.489587,25.491396,7.245030,315.520418,97.113104,7.308734,318.294388,97.968955,36.985075
std,0.006822,0.006822,0.089931,0.041992,0.004228,0.004120,92.84064,0.006939,54.862609,0.239176,...,0.000206,0.000442,0.000454,0.145682,6.344420,1.956144,0.009740,0.423840,0.135408,16.090174
min,29.753597,29.756555,0.482100,2.383500,3.605700,3.605900,112.12400,2.582500,0.209830,0.603600,...,32.067900,25.488600,25.490200,6.878000,299.536000,92.190000,7.283400,317.191000,97.628000,14.000000
25%,29.765546,29.767350,0.780450,2.478800,3.607900,3.608100,287.51950,2.591000,0.212345,0.648950,...,32.068100,25.489500,25.491250,7.236000,315.127500,96.987500,7.302500,318.024000,97.886000,25.500000
50%,29.768302,29.770389,0.814300,2.504100,3.609000,3.609400,341.45200,2.595200,1.234100,0.685100,...,32.068200,25.489700,25.491500,7.297300,317.799000,97.818000,7.308600,318.290000,97.966000,35.000000
75%,29.774870,29.777149,0.838850,2.517750,3.612000,3.612300,394.26450,2.601600,13.511500,0.776150,...,32.068350,25.489900,25.491700,7.328000,319.135000,98.227000,7.315450,318.586500,98.056000,44.500000
max,29.783384,29.785494,0.883600,2.546700,3.621300,3.621500,543.73200,2.610000,304.010000,1.669700,...,32.068800,25.490400,25.492300,7.533600,328.086000,100.977000,7.332100,319.312000,98.302000,95.000000


## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

Two methods are available (if comming from python2 world - ordereddict was important... in py38 a dictionary is inherently ordered)

In [7]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)

FileNotFoundError: [Errno 2] No such file or directory: '../staticdata/cruise_example.yaml'

In [11]:
#Generates an ordereddict but prints better for summary
#likely to be depricated as an ordered dict may not be useful and drops a dependency if its EOL
mooring_config_v2 = load_config.load_config(mooring_meta_file)

In [12]:
mooring_config['Instrumentation'][instrument]

{'InstType': 'SBE-16',
 'SerialNo': '7166',
 'DesignedDepth': 1.0,
 'ActualDepth': 0.0,
 'PreDeploymentNotes': 'UAF',
 'PostDeploymentNotes': '',
 'Deployed': 'y',
 'Recovered': 'y'}

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [13]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time',
  'time_origin': '1900-01-01 00:00:00',
  'units': 'days since 1900-01-01T00:00:00Z'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'timeseries_id': {'cf_role': 'timeseries_id',
  'long_name': 'timeseries id',
  'standard_name': ''},
 'temperature': {'epic_key': 'T_20',
  'generic_name': 'temp',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_wa

In [14]:
#sbe16 data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it
sbe16_wop_data = sbe16_wop_data.rename(columns={'tv290C':'temperature',
                        'sal00':'salinity',
                        'sbeox0Mm/Kg':'oxy_conc',
                        'sbeox0ML/L':'oxy_concM',
                        'sigma-È00':'sigma_theta',
                        'CStarAt0':'Attenuation',
                        'CStarTr0':'Transmittance',
                        'flECO-AFL':'chlor_fluorescence',
                        'empty':'empty', #this will be ignored
                        'flag':'flag'})
sbe16_wop_data.sample()

,timeJV2,temperature,salinity,oxy_conc,oxy_concM,sigma_theta,Attenuation,Transmittance,chlor_fluorescence,flag
date_time,,,,,,,,,,
2016-09-11 15:00:00,129.750255,5.3333,31.7828,320.326,7.3525,25.0887,1.4648,69.3366,10.9974,0.0


In [14]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
sbe16_wop_nc = ncCFsave.EcoFOCI_CFnc_moored(df=sbe16_wop_data, 
                                instrument_yaml=inst_config, 
                                mooring_yaml=mooring_config, 
                                instrument_id=instrument, 
                                inst_shortname=inst_shortname)
sbe16_wop_nc

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [15]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
sbe16_wop_nc.expand_dimensions()

In [16]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
sbe16_wop_nc.variable_meta_data(variable_keys=list(sbe16_wop_data.columns.values),drop_missing=True)
sbe16_wop_nc.temporal_geospatioal_meta_data(depth='designed')
#adding dimension meta needs to come after updating the dimension values... BUG?
sbe16_wop_nc.dimension_meta_data(variable_keys=['depth','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [17]:
#add global attributes
sbe16_wop_nc.deployment_meta_add()
sbe16_wop_nc.get_xdf()

#add instituitonal global attributes
sbe16_wop_nc.institution_meta_add()

#add creation date/time - provenance data
sbe16_wop_nc.provinance_meta_add()

#provide intial qc status field
sbe16_wop_nc.qc_status(qc_status='unknown')


## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [18]:
# combine trim (not mandatory) and filename together (saves to test.nc without name)
sbe16_wop_nc.xarray2netcdf_save(xdf = sbe16_wop_nc.autotrim_time(),
                           filename=sbe16_wop_nc.filename_const(),format="NETCDF3_CLASSIC")

# don't trim the data and pass your own filename
sbe16_wop_nc.xarray2netcdf_save(xdf = sbe16_wop_nc.get_xdf(),
                           filename=sbe16_wop_nc.filename_const(manual_label='test'),format="NETCDF4_CLASSIC")

In [19]:
sbe16_wop_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (depth: 1, latitude: 1, longitude: 1, time: 71)
Coordinates:
  * time                (time) datetime64[ns] 2016-09-07T18:00:00 ... 2016-09...
  * longitude           (longitude) float64 -164.0
  * latitude            (latitude) float64 56.87
  * depth               (depth) float64 1.0
Data variables:
    temperature         (time, depth, latitude, longitude) float64 6.771 ... ...
    salinity            (time, depth, latitude, longitude) float64 0.0032 ......
    oxy_conc            (time, depth, latitude, longitude) float64 368.8 ... ...
    chlor_fluorescence  (time, depth, latitude, longitude) float64 -0.0616 .....
Attributes: (12/13)
    Latitude-Deg_MM.dd_W:                   56 52.1729 N
    Longitude-Deg_MM.dd_N:                  164 02.8716 W
    MooringID:                              16BSM-2A
    platform_deployment_date:               2016-05-05T05:51:00Z
    platform_deployment_cruise_name:        DY1606
    platform_recovery_date:                 2016-09-29T03:45:00Z
    ...                                     ...
    platform_deployment_recovery_comments:  Depth not listed - other two moor...
    WaterDepth:                             71.0
    date_created:                           2021-03-31T12:34:48Z
    date_modified:                          
    QC_indicator:                           unknown
    history:                                Trimmed to deployment.

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history